In [2]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference,faiss]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
cd drive/MyDrive/QA_sentencias_2022/

/content/drive/.shortcut-targets-by-id/14-qpyVS6wHYJKVRbZacpR4r95XmeT0ST/QA_sentencias_2022


In [18]:
ls

 app.ipynb
 config.py
'Copia de 01_Basic_QA_Pipeline.ipynb'
'Copia de 03_Scalable_QA_System.ipynb'
'Copia de 08_Preprocessing.ipynb'
'Copia de 26_Hybrid_Retrieval.ipynb'
'Copia de Tutorial6_Better_Retrieval_via_Embedding_Retrieval.ipynb'
 faiss_document_store.db
 haystack_pipeline_faiss_oai.ipynb
 indexing.ipynb
'No funcionó Prueba retrieve+prompt 22_Pipeline_with_PromptNode.ipynb'
 pasado/
 __pycache__/
 Sentencias_2022/
 Sentencias_2022_impares_1_5_7/
 Sentencias_2022_impares_all/
 Sentencias_2022_pares/


In [19]:
doc_dir='./Sentencias_2022_impares_1_5_7'

In [20]:
doc_dir

'./Sentencias_2022_impares_1_5_7'

In [23]:
import os
from haystack import Pipeline
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import PreProcessor, EmbeddingRetriever
from haystack.utils import convert_files_to_docs
from tqdm import tqdm

def index_documents(doc_dir=doc_dir, document_store_dir=None, db_name='my_index'):
    if document_store_dir is None:
        document_store_dir = os.getcwd()  # Usa el directorio de trabajo actual

    # Verificar si el archivo de índice ya existe en la carpeta de destino
    index_dir = os.path.join(document_store_dir, db_name)
    index_path = os.path.join(index_dir, "my_index.faiss")
    config_path = os.path.join(index_dir, "my_config.json")

    if os.path.exists(index_path):
        print(f"El archivo de índice ya existe en: {index_path}")
        return

    # Asegurarse de que el directorio de destino exista
    if not os.path.exists(document_store_dir):
        os.makedirs(document_store_dir)

    # Asegurarse de que el directorio del índice exista
    if not os.path.exists(index_dir):
        os.makedirs(index_dir)

    # Convertir los documentos en objetos de Haystack
    all_docs = convert_files_to_docs(dir_path=doc_dir)

    # Configurar el preprocesamiento
    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=False,
        split_by="word",
        split_length=100,
    )

    # Configurar el almacenamiento de documentos FAISS
    document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")

    # Configurar el recuperador de incrustaciones
    retriever = EmbeddingRetriever(document_store=document_store,
                                   embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1")

    # Procesar los documentos
    docs = list(tqdm(preprocessor.process(all_docs), desc="Procesando documentos"))

    # Escribir los documentos en el almacenamiento
    document_store.write_documents(docs)

    # Actualizar las incrustaciones
    document_store.update_embeddings(retriever)

    # Guardar el índice y la configuración
    document_store.save(index_path=index_path, config_path=config_path)
    print(f"Índice guardado en: {index_path}")


In [24]:
index_documents()

INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-467-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-471-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-461-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-455-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-457-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-451-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-435A-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-427-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-445-22.txt
INFO:haystack.utils.preprocessing:Converting Sentencias_2022_impares_1_5_7/Copia de T-431-22.txt
INFO:haystack.utils.preproces

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Documents Processed:  44%|████▎     | 10000/22879 [00:58<01:14, 172.09 docs/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Documents Processed:  87%|████████▋ | 20000/22879 [01:53<00:16, 176.18 docs/s]

Batches:   0%|          | 0/90 [00:00<?, ?it/s]

Documents Processed: 30000 docs [02:09, 231.58 docs/s]

Índice guardado en: /content/drive/.shortcut-targets-by-id/14-qpyVS6wHYJKVRbZacpR4r95XmeT0ST/QA_sentencias_2022/my_index/my_index.faiss
